<a href="https://colab.research.google.com/github/Vacayy/ai-playground/blob/main/rag/simple_web_document_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [ ]:
import bs4  # BeautifulSoup4: HTML 및 XML 파일을 파싱하기 위한 라이브러리
from langchain import hub  # LangChain의 다양한 유틸리티와 허브 사용
from langchain_chroma import Chroma  # 벡터 데이터 저장소인 Chroma 사용
from langchain_openai import ChatOpenAI  # OpenAI LLM 모델 사용
from langchain_openai import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain_community.document_loaders import WebBaseLoader  # 웹 문서를 불러오는 유틸리티
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트를 분할하는 도구

In [ ]:
from google.colab import userdata

openai_api_key = userdata.get('openai_api_key')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key) # OpenAI 모델 초기화 (by langchain)

# 크롤링시 적절한 class_를 찾는 방법
1. 브라우저에서 웹페이지 소스 확인:
  - 개발자도구 > 해당 HTML 태그와 클래스 이름을 확인.


2. BeautifulSoup로 임시 파싱 후 클래스 확인:
  - 크롤링한 페이지를 BeautifulSoup로 로드하고 find_all() 메서드로 태그와 클래스 이름 확인.

In [ ]:
"""
WebBaseLoader는 LangChain이 제공하는 유틸리티로, 웹 기반 데이터를 로드하는 데 사용된다.

주요 특징:
- BeautifulSoup을 사용해 HTML 콘텐츠를 파싱한다.
- 클래스 이름이나 태그 등으로 필요한 HTML 요소만 선택적으로 로드할 수 있다.
- web_paths 매개변수에 하나 이상의 URL을 지정해 다양한 페이지에서 데이터를 가져올 수 있다.
"""

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # 크롤링할 웹페이지 URL
    bs_kwargs=dict(  # BeautifulSoup로 원하는 HTML 요소를 추출하는 설정
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")  # 주의. 크롤링 대상 웹문서의 구조에 맞춰서 설정해야 함.
        )
    ),
)
docs = loader.load() # 문서 데이터 불러오기 (크롤링 실행)

In [ ]:
# Retrieval 정확도 향상을 위해 적절한 크기로 Text 분할하기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# 로드한 문서를 분할
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성 (문서를 임베딩 후 Chroma에 저장)
vectorstore = Chroma.from_documents(
    documents=splits,  # 분할된 문서
    embedding=OpenAIEmbeddings(api_key=openai_api_key)  # OpenAI 임베딩 사용
)

print(splits[0], '\n\n===============\n===============\n\n', splits[1])

page_content='LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory' metadata={'sour


## 랭체인의 invoke() 함수
- invoke() 함수는 LangChain의 주요 작업을 수행하는 핵심 메서드 중 하나.
- invoke는 그대로 번역하면 실행한다는 뜻. 어떠한 대상을 실행하는 기능이다.

- 즉, 주어진 입력(프롬프트, 쿼리 등)을 기반으로 모델 또는 리소스에서 작업을 수행하고, 결과를 반환한다.
  - LLM: 입력된 텍스트를 기반으로 AI 모델이 응답을 생성.
  - Retrievers: 사용자의 입력 쿼리를 기반으로 검색 작업 수행.
  - Prompts: 특정 컨텍스트와 질문을 기반으로 프롬프트 생성.


# Prompt 구조

최종 prompt 출력 값을 보면, "**(RAG 사전 prompt) + (Context) + (유저 prompt)**'로 구성되어있다.

유저 질문이 바뀌면 vectorDB에서 레퍼런스 삼을 문서도 다른 걸 검색하기 때문에,
관련 document 내용 prompt도 바뀐다.

In [17]:
# 벡터 저장소에서 검색 가능한 형태의 검색자 초기화
retriever = vectorstore.as_retriever()

# 검색된 문서 포맷팅
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt") # 랭체인 허브에서 사전 정의된 RAG용 프롬프트를 가져옴


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
user_msg = "What is the Self-Reflection?" # 유저 질문
retrieved_docs = retriever.invoke(user_msg) # 유저 질문 기반으로 문서 검색

# 검색된 문서를 컨텍스트로 활용하여 사용자 질문을 포함한 프롬프트 생성
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is the Self-Reflection? \nContext: Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used

# 출력 결과
- 해당 url에 있는 Self-reflection 에 대해서 잘 설명하고 있다.

In [ ]:
response = llm.invoke(user_prompt)
print(response.content)

Self-reflection involves presenting two-shot examples to a Large Language Model (LLM), where each example consists of a failed trajectory and an ideal reflection for improving future plans. Reflections from these examples are then integrated into the agent's working memory, facilitating better context for subsequent queries. This process aims to guide the agent in avoiding past mistakes and enhancing planning efficiency.


# 문서에 없는 내용을 요청한 경우
- 랭체인 사전 Prompt에는 레퍼런스에 없는 것에 대해서는 '모른다'고 답하라고 명시되어 있다.
- 해당 문서에는 없는 '일론 머스크는 누구인가?' 에 대해서 어떻게 답변할까?

<br></br>

### 우선, VectorDB 에서 Context 선정부터 정상 작동하지 않는다.
문서에 있는 Task Decomposition, Self-Reflection 에 대해서 물어봤을 때는 이를 기반으로 벡터 검색을 수행하기 떄문에 연관 내용을 가져왔었다.
<br></br>
그러나 Elon Musk에 대해 물어보자 관련 없는 내용이 선택됐다. 아마 그나마 벡터 유사도가 가장 높은 게 선정되었나보다.

- Task Decomposition: Task decomposition에 대한 세부 정보를 포함한 문서가 선택됨.
- Self-Reflection: Self-reflection과 관련된 Reflexion 프레임워크 문서가 선택됨.
- Elon Musk: 엘론 머스크와 관련 없는 내용(동물의 도구 사용)이 선택됨 → 검색 결과가 적절하지 않음.

<br></br>
그 이유는,
VectorDB는 사용자의 쿼리와 데이터베이스에 저장된 문서(또는 텍스트 청크)의 벡터 표현 간 유사도를 계산하여 가장 관련성 높은 문서를 반환하기 때문이다. 따라서, 검색 과정은 기본적으로 쿼리와 문서 간의 의미적 유사성에 의존한다.

In [19]:
user_msg = "Who is the Elon Musk?"
retrieved_docs = retriever.invoke(user_msg)

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Who is the Elon Musk? \nContext: Fig. 10. A picture of a sea otter using rock to crack open a seashell, while floating in the water. While some other animals can use tools, the complexity is not comparable with humans. (Image source: Animals using tools)\nMRKL (Karpas et al. 2022), short for “Modular Reasoning, Knowledge and Language”, is a neuro-symbolic architecture for autonomous agents. A MRKL system is proposed to contain a collection of “expert” modules and the general-purpose LLM works as a router to route inquiries to the best suitable expert module. These modules can be neural (e.g. deep learning models) or symbolic (e.g. math calculator, currency converter, weather API).\n\nFig. 10. A picture of a se

# 문서에 없는 내용 요청 - 출력 결과
모델은 "I don't know." 라고 솔직하게 답변한 것을 확인할 수 있다.

In [20]:
response = llm.invoke(user_prompt)
print(response.content)

I don't know.
